# Goal

assume we have a set random variables ${Z_1, Z_2, \dots Z_K}$, $Z_i \in \{0,1\}$

We are interested in events of the form: $\mathbf{P}\left(\bigcap_{i \in \mathcal{K}} Z_i = 1\right)$

$\mathcal{K}$ is called itemset

## Definition of association rule

Let $A$ and $B$ be disjoint and complete subsets of $\mathcal{K}$. Then we can have association rule $A \implies B$. A is called antecedent, B -- consequent.

### Metrics for association rules:

**Support**, $T(A \implies B)$ (also the support of the itemset) -- the probability of observing the itemset.

$T(A \implies B) = \mathbf{P}\left(\bigcap_{i \in \mathcal{K}} Z_i = 1\right)$


**Confidence**, $C(A \implies B)$ -- how likely is B, given A.

$C(A \implies B) = \frac{T(A \implies B)}{{T(A)}}$,

where $T(A)$ is defined $\mathbf{P}\left(\bigcap_{i \in \mathcal{A}} Z_i = 1\right)$


The goal is to find assiciation rules with both high support and high confidence.

We are going to estimate support and confidence from a set of observations.

## Method

In order to find those association rules, we first find itemsets with high ($> t, t \in [0;1] $) support. The choice of $t$ depends on the application.

Then, we find association rules that we can construct from those itemsets that also have high ($> c$) confidence

### The Apriori Algorithm

is used to find high-support itemsets.

It requires $t$ to be relatively high (or the data relatively sparse), so that the number of all resulting itemsets is relatively small,

$$|\{\mathcal{K} | T(\mathcal{K}) > t \}| \text{is relatively small}$$

and it is based on the following observation:

If $T(\mathcal{K}) > t$, then $T(\mathcal{A}) > t$ for all $A \subset \mathcal{K}$

The algorithm:

1. Find all itemsets $A$ of size 1, such that $T(\mathcal{A} > t)$
2. Drop all the other items
3. Return to 1, but now form itemsets of size + 1, only from pairs of items that are still present.

Why the requirement for the cardinality to be small?

The number of solution item sets, their size, and the number of passes required over the data can grow exponentially with decreasing size of this lower bound.

One limitation of this algorithm -- it cannot find rules with low support and high confidence (quite valuable).

# Demo

In [110]:
from typing import List, Set, Tuple, Dict
from itertools import combinations
from tqdm.auto import tqdm

FrequentItemset = Tuple[frozenset, float]

def generate_candidates(frequent: Set[frozenset], k: int) -> Set[frozenset]:
    frequent_list = list(frequent)
    candidates = set()
    for i in range(len(frequent_list)):
        for j in range(i + 1, len(frequent_list)):
            union = frequent_list[i] | frequent_list[j]
            if len(union) == k:
                if all(union - {item} in frequent for item in union):
                    candidates.add(union)
    return candidates

# Reduces the number of combinations generated per transaction.
# If an item is not in any frequent itemset, it cannot be part of a larger frequent itemset.
def filter_transactions(transactions: List[frozenset], frequent_items: Set) -> List[frozenset]:
    return [txn & frequent_items for txn in transactions]

def count_supports(
    transactions: List[frozenset], 
    candidates: Set[frozenset], 
    k: int,
    frequent_items: Set = None,
    verbose: bool = False
) -> Dict[frozenset, float]:
    if frequent_items is not None:
        transactions = filter_transactions(transactions, frequent_items)
    counts = {c: 0 for c in candidates}
    for txn in tqdm(transactions, desc="Counting supports (k={})".format(k), disable=not verbose):
        for combo in combinations(txn, k):
            itemset = frozenset(combo)
            if itemset in counts:
                counts[itemset] += 1
    return {itemset: count / len(transactions) for itemset, count in counts.items()}

def apriori(transactions: List[frozenset], min_support: float, verbose: bool = False) -> List[FrequentItemset]:
    all_items = set().union(*transactions)
    candidates = {frozenset([item]) for item in all_items}
    frequent_by_size = {}
    frequent_items = None
    k = 1
    
    while candidates:
        if verbose:
            print(f"Candidates for size {k}: {len(candidates)}")
        supports = count_supports(transactions, candidates, k, frequent_items, verbose)
        frequent = [(itemset, sup) for itemset, sup in supports.items() if sup > min_support]
        if verbose:
            print(f"Frequent itemsets for size {k}: {len(frequent)}")

        frequent_by_size[k] = frequent
        prev_frequent = {itemset for itemset, _ in frequent}
        frequent_items = set().union(*prev_frequent) if prev_frequent else set()
        candidates = generate_candidates(prev_frequent, k + 1)
        k += 1
    
    result = []
    for itemsets in frequent_by_size.values():
        result.extend(itemsets)
    return result


In [107]:
def get_association_rules(
    frequent_itemsets: List[FrequentItemset], 
    min_confidence: float
) -> List[Tuple[frozenset, frozenset, float, float]]:
    rules = []
    itemset_to_support = {itemset: support for itemset, support in frequent_itemsets}
    for itemset, support in frequent_itemsets:
        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                confidence = support / itemset_to_support[antecedent]
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence, support))
    return rules

In [ ]:
# import pandas as pd
# from ucimlrepo import fetch_ucirepo 
# online_retail = fetch_ucirepo(id=352) 
# X = online_retail.data["original"]
# X.head()
# itemsets = X.groupby("InvoiceNo")["StockCode"].apply(frozenset).tolist()
# stock_code_to_description = {row["StockCode"]: row["Description"] for _, row in X.iterrows()}
# print(f"Length of itemsets: {len(itemsets)}")
# print(f"First itemset: {itemsets[0]}")
# print("Descriptions of first itemset:", [stock_code_to_description[code] for code in itemsets[0]])
# contigency_table = (pd.crosstab(X["InvoiceNo"], X["StockCode"]) > 0).astype(int)
# print(f"Contigency table shape: {contigency_table.shape}")
# contigency_table.head()
# cols = contigency_table.columns.tolist()
# transactions = [
#     frozenset(cols[i] for i, val in enumerate(row) if val == 1)
#     for row in contigency_table.values
# ]

In [ ]:
import pandas as pd
import urllib.request
import zipfile
import os

url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
urllib.request.urlretrieve(url, "ml-100k.zip")

with zipfile.ZipFile("ml-100k.zip", "r") as zip_ref:
    zip_ref.extractall(".")

ratings = pd.read_csv(
    "ml-100k/u.data", 
    sep="\t", 
    names=["user_id", "movie_id", "rating", "timestamp"]
)

movies = pd.read_csv(
    "ml-100k/u.item",
    sep="|",
    encoding="latin-1",
    names=["movie_id", "title", "release_date", "video_release", "imdb_url"] + [f"genre_{i}" for i in range(19)]
)

In [116]:
movie_id_to_title = dict(zip(movies["movie_id"], movies["title"]))

In [102]:
liked = ratings[ratings["rating"] >= 4]
transactions = liked.groupby("user_id")["movie_id"].apply(frozenset).tolist()

In [121]:
frequent_itemsets = apriori(transactions, min_support=0.15, verbose=True)
print(f"Found {len(frequent_itemsets)} frequent itemsets")

Candidates for size 1: 1447


Counting supports (k=1):   0%|          | 0/942 [00:00<?, ?it/s]

Frequent itemsets for size 1: 98
Candidates for size 2: 4753


Counting supports (k=2):   0%|          | 0/942 [00:00<?, ?it/s]

Frequent itemsets for size 2: 192
Candidates for size 3: 423


Counting supports (k=3):   0%|          | 0/942 [00:00<?, ?it/s]

Frequent itemsets for size 3: 88
Candidates for size 4: 23


Counting supports (k=4):   0%|          | 0/942 [00:00<?, ?it/s]

Frequent itemsets for size 4: 13
Found 391 frequent itemsets


In [122]:
rules = get_association_rules(frequent_itemsets, min_confidence=0.5)

In [123]:
rules_df = pd.DataFrame(rules, columns=["antecedent", "consequent", "confidence", "support"])
# rules_df["antecedent_description"] = rules_df["antecedent"].apply(lambda x: " ".join([stock_code_to_description[code] for code in x]))
# rules_df["consequent_description"] = rules_df["consequent"].apply(lambda x: " ".join([stock_code_to_description[code] for code in x]))
rules_df["antecedent_description"] = rules_df["antecedent"].apply(lambda x: ", ".join([movie_id_to_title[mid] for mid in x]))
rules_df["consequent_description"] = rules_df["consequent"].apply(lambda x: ", ".join([movie_id_to_title[mid] for mid in x]))

rules_df.sort_values(by="confidence", ascending=False, inplace=True)
rules_df.head()

,antecedent,consequent,confidence,support,antecedent_description,consequent_description
304,"(181, 183)",(50),0.993007,0.150743,"Return of the Jedi (1983), Alien (1979)",Star Wars (1977)
729,"(210, 172, 181)",(50),0.986577,0.156051,"Indiana Jones and the Last Crusade (1989), Emp...",Star Wars (1977)
352,"(181, 173)",(50),0.981595,0.169851,"Return of the Jedi (1983), Princess Bride, The...",Star Wars (1977)
634,"(210, 172, 174)",(50),0.981481,0.168790,"Indiana Jones and the Last Crusade (1989), Emp...",Star Wars (1977)
658,"(172, 173, 174)",(50),0.980645,0.161359,"Empire Strikes Back, The (1980), Princess Brid...",Star Wars (1977)


In [124]:
import plotly.express as px

rules_df["rule"] = rules_df["antecedent_description"] + " → " + rules_df["consequent_description"]

fig = px.scatter(
    rules_df,
    x="support",
    y="confidence",
    hover_data=["rule"],
    title="Association Rules"
)
fig.show()


## Analysis of Results


**Movie Clusters Discovered**:

1. **Star Wars Trilogy** - Empire Strikes Back, Return of the Jedi, Star Wars form an extremely tight cluster. Watching any two almost guarantees the third.

2. **Lucas/Spielberg Universe** - Indiana Jones movies (Raiders, Last Crusade) strongly associate with Star Wars. Makes sense: same creators, same lead actor (Harrison Ford).

3. **Sci-Fi Action** - Terminator, Terminator 2, Alien, Aliens cluster together and connect to Star Wars.

4. **Cult Classics** - Princess Bride, Monty Python and the Holy Grail frequently co-occur.

5. **Prestige Films** - Godfather Part II → Godfather (87% confidence), Shawshank/Schindler's List cluster.

